# Stock Market Prediction 



1.   Amazon - AMZN
2.   Apple - AAPL
3.   Microsoft - MSFT
4. Google - GOOGL
5. Tesla - TSLA
6. Oracle - ORCL


In [ ]:
# Dataset download
import pandas_datareader as pr
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error

step = 70 # time step
company = 'ORCL'
price_type = 'close'
plt.style.use('fivethirtyeight')
plt.figure(figsize=(14,8))
data = pr.get_data_tiingo(company, api_key="0f6351ae343427e511f4d085681db7e303ffb969")
data.to_csv(company+'.csv')

data = pd.read_csv(company+'.csv')

# Baseline calculation

Base_errors = []

for types in ['open','close','high','low']:

  Price = data[types]
  Means = list()

  for i in range(len(Price) - step):
    Means.append(sum(Price[i:i+step])/step)  

  Error = math.sqrt(mean_squared_error(Price[step:],Means))
  Base_errors.append(Error)   
  print("Base case error for",types,"=",Error)    
  plt.plot(Price,label=types)
plt.legend()
Price = data[price_type]


In [ ]:
# MinMax Scalimg
import numpy as np
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler(feature_range=(0,1))
Price = mms.fit_transform(np.array(Price).reshape(-1,1))

# Dataset split into train and test sets

train_size = int(len(Price)*0.80)
test_size = len(Price)-train_size
train_data,test_data = Price[0:train_size,:],Price[train_size:len(Price),:1]
train_size, test_size

In [ ]:
# Converting data to timeseries input

def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return np.array(dataX), np.array(dataY)

time_step = step
trainX, trainY = create_dataset(train_data, time_step)
testX, testY = create_dataset(test_data, time_step)

print(trainX.shape), print(trainY.shape)
print(testX.shape), print(testY.shape)

trainX = trainX.reshape(trainX.shape[0],trainX.shape[1] , 1)
testX = testX.reshape(testX.shape[0],testX.shape[1] , 1)

In [ ]:
# Stacked LSTM model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

lstm = Sequential()
lstm.add(LSTM(128, return_sequences=True, input_shape= (step, 1)))
lstm.add(LSTM(64, return_sequences=False))
lstm.add(Dense(25))
lstm.add(Dense(1))
lstm.compile(optimizer='adam', loss='mean_squared_error')
lstm.summary()

In [ ]:
lstm.fit(trainX,trainY,validation_data=(testX,testY),epochs=100,batch_size=64,verbose=1)

In [ ]:
# Simple RNN

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dense

rnn = Sequential()
rnn.add(LSTM(128, return_sequences=True, input_shape= (step, 1)))
rnn.add(LSTM(64, return_sequences=False))
rnn.add(Dense(25))
rnn.add(Dense(1))
rnn.compile(optimizer='adam', loss='mean_squared_error')
rnn.summary()

In [ ]:
rnn.fit(trainX,trainY,validation_data=(testX,testY),epochs=100,batch_size=64,verbose=1)

In [ ]:
# GRU

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

gru = Sequential()
gru.add(LSTM(128, return_sequences=True, input_shape= (step, 1)))
gru.add(LSTM(64, return_sequences=False))
gru.add(Dense(25))
gru.add(Dense(1))
gru.compile(optimizer=SGD(lr=0.01, decay=1e-7, momentum=0.9, nesterov=False),loss='mean_squared_error')
gru.summary()

In [ ]:
gru.fit(trainX,trainY,validation_data=(testX,testY),epochs=100,batch_size=64,verbose=1)

In [ ]:
# Prediction and error calculation for lstm

train_predict_lstm = lstm.predict(trainX)
test_predict_lstm = lstm.predict(testX)

train_predict_rnn = rnn.predict(trainX)
test_predict_rnn = rnn.predict(testX)

train_predict_gru = gru.predict(trainX)
test_predict_gru = gru.predict(testX)



print("Shape of LSTM test",test_predict_lstm.shape)
print("Shape of RNN test",test_predict_rnn.shape)
print("Shape of GRU test",test_predict_gru.shape)

testY_lstm = testY.reshape(test_size-step-1,1)
trainY_lstm = trainY.reshape(train_size-step-1,1)

testY_rnn = testY.reshape(test_size-step-1,1)
trainY_rnn = trainY.reshape(train_size-step-1,1)

testY_gru = testY.reshape(test_size-step-1,1)
trainY_gru = trainY.reshape(train_size-step-1,1)

testY_lstm = mms.inverse_transform(testY_lstm)
trainY_lstm = mms.inverse_transform(trainY_lstm)

testY_rnn = mms.inverse_transform(testY_rnn)
trainY_rnn = mms.inverse_transform(trainY_rnn)

testY_gru = mms.inverse_transform(testY_gru)
trainY_gru = mms.inverse_transform(trainY_gru)

print(testY_lstm[0])
print(trainY_lstm[0])

train_predict_lstm = mms.inverse_transform(train_predict_lstm)
test_predict_lstm = mms.inverse_transform(test_predict_lstm)
train_predict_rnn = mms.inverse_transform(train_predict_rnn)
test_predict_rnn = mms.inverse_transform(test_predict_rnn)
train_predict_gru = mms.inverse_transform(train_predict_gru)
test_predict_gru = mms.inverse_transform(test_predict_gru)

print(train_predict_lstm[0])
print(test_predict_lstm[0])

print("Train data error for LSTM model =",math.sqrt(mean_squared_error(trainY_lstm,train_predict_lstm)))
print("Test data error for LSTM model=",math.sqrt(mean_squared_error(testY_lstm,test_predict_lstm)))
print("----------------------------------------------------------------------")
print("Train data error for RNN model =",math.sqrt(mean_squared_error(trainY_rnn,train_predict_rnn)))
print("Test data error for RNN model=",math.sqrt(mean_squared_error(testY_rnn,test_predict_rnn)))
print("----------------------------------------------------------------------")
print("Train data error for GRU model =",math.sqrt(mean_squared_error(trainY_gru,train_predict_gru)))
print("Test data error for GRU model=",math.sqrt(mean_squared_error(testY_gru,test_predict_gru)))

plt.figure(figsize=(14,8))
plt.title("Different Models")
plt.plot(test_predict_lstm,label = "predict_LSTM")
plt.plot(testY_lstm,label = "original")
plt.plot(test_predict_rnn,label = "predict_SimpleRNN")
plt.plot(test_predict_gru,label = "predict_GRU")
plt.legend()